# BSAD - Preparación para Entrevista Técnica

**Proyecto:** Bayesian Security Anomaly Detection  
**Objetivo:** Detección de anomalías en datos de seguridad usando modelado Bayesiano jerárquico

---

## Índice

1. [Stack Técnico y Librerías](#1-stack-técnico)
2. [El Problema que Resuelve BSAD](#2-el-problema)
3. [Modelo Matemático](#3-modelo-matemático)
4. [Pipeline End-to-End](#4-pipeline)
5. [Métricas y Evaluación](#5-métricas)
6. [Módulo Triage (Operacional)](#6-triage)
7. [Preguntas de Entrevista](#7-preguntas)
8. [Código Ejecutable](#8-código)

---
## 1. Stack Técnico y Librerías <a id='1-stack-técnico'></a>

### Core Bayesiano
| Librería | Versión | Propósito |
|----------|---------|----------|
| **PyMC** | 5.10+ | Framework de programación probabilística. Usa MCMC (HMC/NUTS) para inferencia |
| **ArviZ** | 0.17+ | Diagnósticos MCMC, visualización de posteriors, formato InferenceData |
| **PyTensor** | 2.18+ | Backend de cómputo tensorial (reemplaza Theano) |

### Data Science
| Librería | Propósito |
|----------|----------|
| **NumPy** | Arrays numéricos, operaciones vectorizadas |
| **Pandas** | DataFrames, manipulación de datos tabulares |
| **SciPy** | Distribuciones estadísticas (NegBinomial), funciones especiales (logsumexp) |
| **Scikit-learn** | Métricas (PR-AUC, ROC-AUC), train/test split |

### Visualización & CLI
| Librería | Propósito |
|----------|----------|
| **Matplotlib/Seaborn** | Gráficos estáticos |
| **Typer** | CLI framework (comandos `bsad train`, `bsad score`, etc.) |
| **Rich** | Formateo de tablas en terminal |

### I/O
| Librería | Propósito |
|----------|----------|
| **PyArrow** | Lectura/escritura eficiente de Parquet |
| **NetCDF4** | Almacenamiento de traces MCMC (via ArviZ) |

---
## 2. El Problema que Resuelve BSAD <a id='2-el-problema'></a>

### Características de los datos de seguridad

```
┌─────────────────────────────────────────────────────────────┐
│  DATOS DE SEGURIDAD típicos:                                │
│                                                             │
│  • Conteos (enteros): intentos de login, requests, bytes   │
│  • Estructura entidad-tiempo: user_id × día                │
│  • Eventos RAROS: <5% son ataques (class imbalance extremo)│
│  • Overdispersion: Varianza >> Media                       │
│  • Entidades heterogéneas: algunos users muy activos,      │
│    otros casi inactivos                                    │
└─────────────────────────────────────────────────────────────┘
```

### Por qué los métodos clásicos fallan

| Método | Problema con datos de seguridad |
|--------|--------------------------------|
| **Z-score global** | Ignora que cada entidad tiene su propio baseline |
| **Poisson** | Asume var = mean; security data tiene var >> mean |
| **Random Forest / XGBoost** | No da incertidumbre, requiere feature engineering manual, sufre con class imbalance |
| **Isolation Forest** | No modela estructura entidad-tiempo |

### La solución BSAD

```
┌─────────────────────────────────────────────────────────────┐
│  BSAD usa:                                                  │
│                                                             │
│  1. Negative Binomial → maneja overdispersion              │
│  2. Hierarchical model → cada entidad tiene su θ[e]        │
│  3. Partial pooling → entidades sparse "toman prestado"    │
│     de la población                                        │
│  4. Full Bayesian → incertidumbre cuantificada             │
└─────────────────────────────────────────────────────────────┘
```

---
## 3. Modelo Matemático <a id='3-modelo-matemático'></a>

### Estructura Jerárquica (3 niveles)

```
╔══════════════════════════════════════════════════════════════╗
║  NIVEL 1: Hiperpriors Poblacionales                         ║
║  ─────────────────────────────────────────                  ║
║  μ ~ Exponential(λ = 0.1)    # Tasa media global            ║
║  α ~ HalfNormal(σ = 2.0)     # Shape/overdispersion         ║
╠══════════════════════════════════════════════════════════════╣
║  NIVEL 2: Tasas por Entidad (Partial Pooling)               ║
║  ─────────────────────────────────────────────              ║
║  θ[e] ~ Gamma(shape=α·μ, rate=α)  para cada entidad e      ║
║                                                              ║
║  → E[θ[e]] = μ  (centrado en media poblacional)             ║
║  → Var[θ[e]] = μ/α (controlado por α)                       ║
╠══════════════════════════════════════════════════════════════╣
║  NIVEL 3: Observaciones                                      ║
║  ─────────────────────────                                  ║
║  φ ~ HalfNormal(σ = 2.0)     # Dispersión observacional     ║
║  y[i] ~ NegBinomial(μ=θ[entity[i]], α=φ)                    ║
║                                                              ║
║  donde: Var(y) = μ + μ²/φ  (overdispersion cuando φ < ∞)    ║
╚══════════════════════════════════════════════════════════════╝
```

### Diagrama Visual

```
                    ┌───────────┐
                    │  μ, α     │  ← Hiperparámetros poblacionales
                    └─────┬─────┘
                          │
          ┌───────────────┼───────────────┐
          ▼               ▼               ▼
      ┌───────┐       ┌───────┐       ┌───────┐
      │ θ[1]  │       │ θ[2]  │  ...  │ θ[E]  │  ← Tasas por entidad
      └───┬───┘       └───┬───┘       └───┬───┘
          │               │               │
    ┌─────┼─────┐   ┌─────┼─────┐   ┌─────┼─────┐
    ▼     ▼     ▼   ▼     ▼     ▼   ▼     ▼     ▼
  y[1,1] y[1,2]... y[2,1] y[2,2]... y[E,1] y[E,2]...  ← Observaciones
```

### Partial Pooling: La Magia

```
┌────────────────────────────────────────────────────────────────┐
│                                                                │
│  NO POOLING          PARTIAL POOLING         COMPLETE POOLING │
│  ──────────          ───────────────         ──────────────── │
│  Cada entidad        Balance automático      Todas las        │
│  independiente       según datos             entidades iguales│
│                                                                │
│  θ[e] estimado       θ[e] = weighted avg     θ[e] = μ global  │
│  solo con datos      de datos propios +      para todos       │
│  de entidad e        prior poblacional                        │
│                                                                │
│  PROBLEMA:           SOLUCIÓN BSAD:          PROBLEMA:        │
│  Alta varianza       Shrinkage adaptativo    Alto sesgo       │
│  para sparse         hacia μ                 ignora diferencias│
│                                                                │
└────────────────────────────────────────────────────────────────┘
```

### Anomaly Score

```python
# Para cada observación y[i]:
anomaly_score[i] = -log P(y[i] | posterior)

# Donde P(y|posterior) se calcula promediando sobre muestras MCMC:
log P(y) = logsumexp([log P(y | θ_s, φ_s) for s in samples]) - log(n_samples)
```

**Interpretación:**
- Score ALTO = probabilidad BAJA bajo el modelo = ANÓMALO
- Score BAJO = probabilidad ALTA = comportamiento normal

---
## 3.5 Diagrama de Arquitectura Completa <a id='3.5-arquitectura'></a>

```mermaid
flowchart TB
    subgraph DATA["📊 1. DATOS"]
        RAW[("Eventos Crudos<br/>user_id, timestamp,<br/>event_count")]
        VALID["Validar:<br/>✓ Conteos<br/>✓ Entidades<br/>✓ Var >> Mean<br/>✓ <5% ataques"]
        RAW --> VALID
    end

    subgraph FEATURES["⚙️ 2. FEATURES"]
        AGG["GROUP BY<br/>(entidad, ventana)"]
        SPLIT["Split Temporal<br/>Train | Test"]
        AGG --> SPLIT
    end

    subgraph MODEL["🧠 3. MODELO JERÁRQUICO"]
        direction TB
        POP["🌐 Población<br/>μ, α"]
        ENT["👥 Entidad<br/>θ[e] ~ Gamma(μα, α)"]
        OBS["📈 Observación<br/>y ~ NegBin(θ[e], φ)"]
        POP -->|"Partial Pooling"| ENT --> OBS
    end

    subgraph SCORE["📊 4. SCORING"]
        CALC["score = -log P(y|posterior)"]
        UNCERT["± incertidumbre<br/>(std, CI)"]
        CALC --> UNCERT
    end

    subgraph TRIAGE["🚨 5. TRIAGE"]
        RISK["Risk = 0.5×score<br/>+ 0.3×confianza<br/>+ 0.2×novedad"]
        ALERT["Alertas<br/>priorizadas"]
        RISK --> ALERT
    end

    DATA ==> FEATURES ==> MODEL ==>|"MCMC<br/>2000 samples"| SCORE ==> TRIAGE

    style MODEL fill:#fff3e0
    style SCORE fill:#fce4ec
    style TRIAGE fill:#fff8e1
```

### Flujo en Una Línea

```
Eventos → Agregar por entidad → Modelo Bayesiano → Score (-log P) → Risk Score → Alertas TOP-K
```

### Variables Clave en Cada Etapa

| Etapa | Variables | Dimensiones |
|-------|-----------|-------------|
| **Input** | `event_count`, `entity_id`, `timestamp` | N observaciones |
| **Model** | `y[n]`, `entity_idx[n]`, `n_entities` | Arrays para PyMC |
| **Posterior** | `θ[e]` (tasa), `φ` (dispersión), `μ`, `α` | S muestras × E entidades |
| **Score** | `anomaly_score`, `score_std`, `CI_lower`, `CI_upper` | N observaciones |
| **Risk** | `risk_score`, `rank`, `threshold` | N observaciones |

---
## 4. Pipeline End-to-End <a id='4-pipeline'></a>

### Flujo de Datos

```
┌─────────────────┐
│  Raw Events     │  (timestamp, user_id, ip, endpoint, status, bytes...)
└────────┬────────┘
         │
         ▼  build_features()
┌─────────────────┐
│  modeling_df    │  Agregado por (entity, window): event_count, unique_ips...
└────────┬────────┘
         │
         ▼  get_model_arrays()
┌─────────────────┐
│  arrays dict    │  {y: ndarray, entity_idx: ndarray, n_entities: int}
└────────┬────────┘
         │
         ▼  train_model()  [MCMC - LENTO]
┌─────────────────┐
│  trace          │  ArviZ InferenceData con posteriors de μ, α, θ[e], φ
└────────┬────────┘
         │
         ▼  compute_scores()
┌─────────────────┐
│  scores dict    │  {anomaly_score, score_std, score_lower, score_upper}
└────────┬────────┘
         │
         ▼  create_scored_df()
┌─────────────────┐
│  scored_df      │  DataFrame con scores, ranks, intervalos, ground truth
└────────┬────────┘
         │
         ▼  evaluate()
┌─────────────────┐
│  metrics        │  {pr_auc, roc_auc, precision_at_k, recall_at_k...}
└─────────────────┘
```

### Pasos del Pipeline en Código

| Paso | Función | Input | Output |
|------|---------|-------|--------|
| 1 | `generate_data()` | Settings | (events_df, attacks_df) |
| 2 | `build_features()` | events_df | (modeling_df, metadata) |
| 3 | `get_model_arrays()` | modeling_df | dict {y, entity_idx, ...} |
| 4 | `train_model()` | arrays, settings | trace (InferenceData) |
| 5 | `compute_scores()` | y, trace, entity_idx | scores dict |
| 6 | `compute_intervals()` | trace, entity_idx | intervals dict |
| 7 | `create_scored_df()` | modeling_df, scores | scored_df |
| 8 | `evaluate()` | scored_df | metrics dict |
| 9 | `create_plots()` | scored_df, metrics | plot paths |

---
## 5. Métricas y Evaluación <a id='5-métricas'></a>

### Métricas de Clasificación

| Métrica | Fórmula | Cuándo Usar |
|---------|---------|-------------|
| **PR-AUC** | Área bajo Precision-Recall curve | **PRIMARIA** para eventos raros |
| **ROC-AUC** | Área bajo ROC curve | Secundaria; puede ser engañosa con imbalance |
| **Precision@k** | TP en top-k / k | "De mis top 50 alertas, ¿cuántas son reales?" |
| **Recall@k** | TP en top-k / total_positivos | "¿Qué % de ataques capturo en top 50?" |

### ¿Por qué PR-AUC > ROC-AUC para eventos raros?

```
Ejemplo con 2% attack rate (98 normales, 2 ataques):

Modelo "predice todo normal":
├── ROC-AUC: Puede ser ~0.50 (parece "random")
└── PR-AUC: ~0.02 (refleja el fracaso real)

Modelo mediocre que detecta 1 de 2 ataques con 10 FP:
├── ROC-AUC: ~0.95 (parece excelente!)
│   └── Porque TNR = 88/98 = 0.90 domina
└── PR-AUC: ~0.15 (refleja la realidad operacional)
    └── Precision = 1/11 = 0.09

CONCLUSIÓN: PR-AUC es más honesto con class imbalance.
```

### Métricas MCMC (Diagnósticos)

| Métrica | Valor OK | Interpretación |
|---------|----------|----------------|
| **R-hat** | < 1.05 | Convergencia entre cadenas |
| **ESS (bulk)** | > 400 | Muestras efectivas independientes |
| **Divergences** | 0 | Problemas de geometría del posterior |

```python
# Cómo verificar en código:
import arviz as az
summary = az.summary(trace, var_names=["mu", "alpha", "phi"])
print(summary[["r_hat", "ess_bulk"]])
```

---
## 6. Módulo Triage (Operacional) <a id='6-triage'></a>

### Componentes

```
src/triage/
├── risk_score.py           # Score compuesto
├── calibrate_thresholds.py # Calibración de alertas
├── ranking_metrics.py      # Métricas operacionales
└── entity_context.py       # Enriquecimiento de alertas
```

### 1. RiskScorer: Score Compuesto

```python
risk = 0.5 * normalized_score  +  # Magnitud del score
       0.3 * confidence        +  # 1/(1+std) - inverso de incertidumbre
       0.2 * novelty              # Entidades nuevas = más riesgo
```

**Por qué:** El score raw no es suficiente. Una anomalía con alta incertidumbre es menos confiable.

### 2. AlertBudget: Calibración de Threshold

```
┌────────────────────────────────────────────────────────────┐
│  MODO                 PREGUNTA DEL SOC                    │
├────────────────────────────────────────────────────────────┤
│  fixed_alerts    "Solo puedo manejar 50 alertas/día"     │
│                  → Calcula threshold para generar ≤50     │
├────────────────────────────────────────────────────────────┤
│  fixed_recall    "Quiero detectar al menos 30% de        │
│                   ataques"                                │
│                  → Calcula threshold para recall ≥ 0.30   │
├────────────────────────────────────────────────────────────┤
│  fixed_fpr       "Tolero máximo 5% de falsos positivos"  │
│                  → Calcula threshold para FPR ≤ 0.05      │
└────────────────────────────────────────────────────────────┘
```

### 3. EntityContext: Enriquecimiento

```
┌─────────────────────────────────────────────────────────────┐
│  ALERTA ENRIQUECIDA para el analista:                      │
│                                                             │
│  Entity: user_0234                                         │
│  Current value: 127 events (baseline: 45 ± 12)             │
│  Deviation: 6.8σ above baseline (extremely high)           │
│  Global percentile: 99.2%                                  │
│  Previous alerts for this entity: 3                        │
│  Confidence: HIGH (narrow uncertainty bounds)              │
└─────────────────────────────────────────────────────────────┘
```

---
## 7. Preguntas de Entrevista <a id='7-preguntas'></a>

### Fundamentos del Modelo

---

**Q1: ¿Por qué Negative Binomial en lugar de Poisson?**

> Poisson asume que varianza = media. En datos de seguridad, la varianza típicamente **excede** la media (overdispersion). 
> 
> Negative Binomial tiene un parámetro extra (φ) que modela esta dispersión adicional:
> - `Var(y) = μ + μ²/φ`
> - Cuando φ → ∞, NegBin → Poisson
> - Valores finitos de φ capturan overdispersion

---

**Q2: ¿Qué es partial pooling y por qué es importante?**

> Es un balance entre dos extremos:
> - **No pooling**: Cada entidad se estima independientemente → alta varianza para entidades con pocos datos
> - **Complete pooling**: Todas las entidades comparten el mismo parámetro → sesgo, ignora diferencias
>
> **Partial pooling** (lo que hace BSAD):
> - Entidades con **pocos datos** → su θ[e] se "encoge" hacia la media poblacional μ
> - Entidades con **muchos datos** → su θ[e] se estima principalmente de sus propios datos
> - El modelo **aprende automáticamente** cuánto confiar en cada fuente

---

**Q3: ¿Por qué PR-AUC sobre ROC-AUC para eventos raros?**

> Con 2% attack rate:
> - ROC-AUC puede ser 0.95+ con un modelo mediocre porque **True Negatives dominan**
> - PR-AUC **penaliza los falsos positivos** más severamente
> - PR-AUC refleja mejor la realidad operacional: "¿cuántas de mis alertas son reales?"

---

**Q4: ¿Cómo se calcula el anomaly score?**

> ```
> score = -log P(y | posterior)
> ```
> 
> Pasos:
> 1. Para cada muestra MCMC s, calcular P(y | θ_s, φ_s) usando la PMF de NegBinomial
> 2. Promediar en log-space: `logsumexp(log_probs) - log(n_samples)`
> 3. Negar: score alto = probabilidad baja = anómalo

---

### Diagnósticos y Convergencia

---

**Q5: ¿Qué indica R-hat y cuál es un valor aceptable?**

> R-hat (Gelman-Rubin statistic) mide si **múltiples cadenas MCMC convergieron** a la misma distribución.
> - Compara varianza **within-chain** vs **between-chain**
> - R-hat ≈ 1.0 → cadenas están mezcladas, convergieron
> - **Aceptable: < 1.05** (algunos usan < 1.01)
> - R-hat > 1.1 → problema serio, no confiar en resultados

---

**Q6: ¿Qué es ESS y por qué importa?**

> **Effective Sample Size**: MCMC genera muestras **autocorrelacionadas**. ESS estima cuántas muestras **independientes** equivalentes tienes.
> 
> - Si tienes 2000 muestras pero ESS = 200, es como tener solo 200 muestras independientes
> - **Mínimo recomendado: 400** para estimaciones estables
> - ESS bajo indica: necesitas más samples, o el modelo tiene problemas de muestreo

---

**Q7: ¿Qué son las divergencias en HMC?**

> Son **fallos numéricos** durante el muestreo Hamiltoniano.
> - HMC simula una partícula moviéndose en el espacio de parámetros
> - Divergencia = la simulación "explota" (error de integración)
> - Indica **geometría problemática** del posterior (curvaturas extremas)
> - **Deberían ser 0**. Si hay muchas → reparametrizar modelo o ajustar sampler

---

### Diseño y Trade-offs

---

**Q8: ¿Cómo maneja el modelo entidades nuevas (cold start)?**

> Gracias al **partial pooling**:
> - Entidad nueva → prior muy fuerte hacia μ (media poblacional)
> - Primeras observaciones → θ[e] cercano a μ
> - A medida que acumula datos → θ[e] se individualiza
> 
> No hay "cold start" catastrófico porque siempre hay un prior informativo.

---

**Q9: ¿Por qué no usar Random Forest o XGBoost directamente?**

> | Aspecto | Classifiers | BSAD |
> |---------|-------------|------|
> | Incertidumbre | No (o requiere calibración extra) | Sí, intervalos credibles |
> | Class imbalance | Requiere SMOTE, class weights | Manejo natural |
> | Estructura entidad | Feature engineering manual | Modelado explícito |
> | Sparse entities | Problemas | Partial pooling |
> | Interpretabilidad | Caja negra | Parámetros interpretables |

---

**Q10: ¿Cómo escala este enfoque?**

> **Entrenamiento**: O(n_samples × n_observations × n_entities)
> - LENTO (horas para modelos realistas)
> - Se hace **offline**, no en tiempo real
> 
> **Scoring**: O(n_observations × n_posterior_samples)
> - RÁPIDO (lookup + aritmética)
> - Apto para batch o near-real-time
> 
> **Estrategia de producción**:
> - Reentrenar modelo diario/semanal
> - Usar modelo entrenado para scoring continuo

---

### Operacional

---

**Q11: ¿Cómo calibras el threshold para alertas?**

> Tres estrategias en `AlertBudget`:
> 1. **Fixed alerts**: "Puedo manejar 50 alertas/día" → encuentra threshold
> 2. **Fixed recall**: "Quiero detectar 30% de ataques" → encuentra threshold
> 3. **Fixed FPR**: "Tolero 5% false positive rate" → encuentra threshold
> 
> Cada modo tiene trade-offs diferentes entre workload y coverage.

---

**Q12: ¿Qué tipos de ataques puede detectar BSAD?**

> **Detecta bien:**
> - Brute force (conteo anómalo de intentos)
> - Credential stuffing (patrones de múltiples usuarios)
> - Data exfiltration (volumen de bytes inusual)
> - Beaconing/C2 (patrones periódicos)
> 
> **NO detecta:**
> - Ataques de contenido (SQLi, XSS) - no modela contenido
> - Zero-days que no afectan conteos
> - Ataques "low and slow" que se mantienen bajo el radar

---

**Q13: ¿Cuáles son las limitaciones principales?**

> 1. **Solo datos de conteo** - no features continuos multivariados
> 2. **Requiere estructura entidad-tiempo** - no apto para datos sin entidades
> 3. **Entrenamiento lento** - MCMC toma horas
> 4. **No real-time** - latencia > 100ms
> 5. **Requiere ground truth** para calibrar thresholds (aunque no para entrenar)

---

**Q14: ¿Cómo explicarías una alerta a un analista?**

> "El usuario X generó 127 eventos en esta ventana. Su baseline histórico es 45±12 eventos. 
> Esto es 6.8 desviaciones estándar por encima de lo normal para este usuario específico.
> El modelo asigna una probabilidad muy baja a este comportamiento, lo que resulta en un 
> anomaly score en el percentil 99.2. Además, este usuario ha generado 3 alertas previas."

---

**Q15: ¿Qué harías si el modelo tiene muchas divergencias?**

> 1. **Aumentar target_accept** (ej: 0.95 → 0.99) - pasos más pequeños
> 2. **Reparametrizar** - usar non-centered parameterization para θ[e]
> 3. **Verificar priors** - priors muy amplios pueden causar problemas
> 4. **Revisar datos** - outliers extremos pueden distorsionar el posterior
> 5. **Simplificar modelo** - quizás demasiados parámetros para los datos

---
## 7.5 Benchmark y Robustez (NUEVO) <a id='7.5-benchmark'></a>

### Benchmark Reproducible

BSAD se compara contra baselines específicos para datos de conteo:

| Modelo | PR-AUC | ROC-AUC | Recall@50 | Tipo |
|--------|--------|---------|-----------|------|
| **BSAD** | 0.562 | 0.943 | 1.000 | Bayesiano Jerárquico |
| NB_EmpBayes | 0.568 | 0.954 | 1.000 | Específico conteos |
| GLMM_NB | 0.567 | 0.952 | 1.000 | Específico conteos |
| NB_MLE | 0.466 | 0.856 | 0.800 | Específico conteos |

**Insight**: BSAD y NB_EmpBayes rinden similar porque ambos usan partial pooling. La ventaja de BSAD es la cuantificación completa de incertidumbre.

### Análisis de Robustez

| Test | Resultado | Implicación |
|------|-----------|-------------|
| **Sensibilidad Tasa Ataque** | PR-AUC: 0.46 (0.5%) → 0.89 (10%) | Diseñado para <5% |
| **Deriva Temporal** | Sin degradación (+7.7% en test) | Seguro para producción |
| **Arranque en Frío** | -14% para entidades nuevas | Aceptable con partial pooling |
| **Estabilidad Ranking** | Spearman ρ=0.51 | Reentrenamiento semanal |

### Preguntas de Entrevista sobre Benchmark

---

**Q16: ¿Cómo validaste que tu modelo funciona mejor que las alternativas?**

> Creé un benchmark reproducible comparando BSAD contra:
> 1. **Baselines justos** (específicos para conteos): NB_MLE, NB_EmpBayes, GLMM_NB
> 2. **Detectores genéricos** (referencia): Isolation Forest, One-Class SVM, LOF
>
> Resultado: BSAD tiene rendimiento similar a NB_EmpBayes (~0.56 PR-AUC), lo cual es esperado porque ambos usan partial pooling. La ventaja de BSAD es la **cuantificación de incertidumbre completa**.

---

**Q17: ¿Cómo probaste la robustez del modelo?**

> Ejecuté 4 tests de robustez:
> 1. **Sensibilidad a tasa de ataque**: PR-AUC escala de 0.46 (0.5%) a 0.89 (10%)
> 2. **Deriva temporal**: Sin degradación en datos futuros (+7.7% en test vs train)
> 3. **Arranque en frío**: -14% para entidades no vistas (aceptable con partial pooling)
> 4. **Estabilidad de ranking**: Spearman ρ=0.51 (moderada, sugiere reentrenamiento semanal)

---

**Q18: ¿Por qué BSAD no gana siempre en ROC-AUC?**

> Porque estamos comparando manzanas con naranjas:
> - **Random Forest** usa labels (supervisado)
> - **BSAD** es no supervisado
>
> La comparación justa es en **métricas operacionales**: BSAD genera 8-14× menos alertas que RF para el mismo recall. Eso es lo que importa en un SOC.

---
## 8. Código Ejecutable <a id='8-código'></a>

Ejecuta las siguientes celdas para ver el modelo en acción.

In [ ]:
# Setup
import sys
sys.path.insert(0, '../src')

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from bsad.config import Settings
from bsad import steps

print("✓ Imports OK")

In [ ]:
# Configuración para demo rápido
settings = Settings(
    n_entities=50,      # Pocas entidades para demo
    n_days=14,          # 2 semanas
    attack_rate=0.03,   # 3% ataques
    n_samples=500,      # Pocas muestras MCMC (solo demo)
    n_tune=300,
    n_chains=2,
    random_seed=42
)

print(f"Configuración:")
print(f"  - Entidades: {settings.n_entities}")
print(f"  - Días: {settings.n_days}")
print(f"  - Attack rate: {settings.attack_rate:.1%}")
print(f"  - MCMC samples: {settings.n_samples}")

In [ ]:
# Paso 1: Generar datos sintéticos
print("Generando datos sintéticos...")
events_df, attacks_df = steps.generate_data(settings)

print(f"\n✓ Eventos generados: {len(events_df):,}")
print(f"✓ Ataques inyectados: {len(attacks_df)}")
print(f"\nTipos de ataque:")
print(attacks_df['attack_type'].value_counts())

In [ ]:
# Paso 2: Feature engineering
print("Construyendo features...")
modeling_df, metadata = steps.build_features(events_df, attacks_df, settings)

print(f"\n✓ Observaciones (entity-windows): {len(modeling_df):,}")
print(f"✓ Entidades únicas: {metadata['n_entities']}")
print(f"✓ Attack rate: {metadata['attack_rate']:.2%}")

print(f"\nColumnas del modeling_df:")
print(modeling_df.columns.tolist())

In [ ]:
# Visualizar distribución de conteos (overdispersion)
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Histograma de event_count
axes[0].hist(modeling_df['event_count'], bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(modeling_df['event_count'].mean(), color='red', linestyle='--', label=f"Mean: {modeling_df['event_count'].mean():.1f}")
axes[0].axvline(modeling_df['event_count'].std(), color='orange', linestyle=':', label=f"Std: {modeling_df['event_count'].std():.1f}")
axes[0].set_xlabel('Event Count')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribución de Conteos')
axes[0].legend()

# Overdispersion check
mean_count = modeling_df['event_count'].mean()
var_count = modeling_df['event_count'].var()
axes[1].bar(['Mean', 'Variance'], [mean_count, var_count], color=['steelblue', 'coral'])
axes[1].set_title(f'Overdispersion Check\nVar/Mean = {var_count/mean_count:.2f}')
axes[1].set_ylabel('Value')

plt.tight_layout()
plt.show()

print(f"\n📊 Overdispersion ratio (Var/Mean): {var_count/mean_count:.2f}")
print(f"   Si fuera Poisson, este ratio sería ≈ 1.0")
print(f"   Ratio > 1 indica overdispersion → justifica Negative Binomial")

In [ ]:
# Paso 3: Extraer arrays para PyMC
arrays = steps.get_model_arrays(modeling_df)

print("Arrays para el modelo:")
for key, val in arrays.items():
    if isinstance(val, np.ndarray):
        print(f"  {key}: shape={val.shape}, dtype={val.dtype}")
    else:
        print(f"  {key}: {val}")

In [ ]:
# Paso 4: Entrenar modelo (MCMC)
print("Entrenando modelo Bayesiano...")
print("(Esto puede tomar 1-2 minutos con la configuración de demo)\n")

trace = steps.train_model(arrays, settings)

print("\n✓ Modelo entrenado!")

In [ ]:
# Diagnósticos MCMC
import arviz as az

print("📊 Diagnósticos MCMC:\n")
summary = az.summary(trace, var_names=["mu", "alpha", "phi"])
print(summary[["mean", "sd", "r_hat", "ess_bulk"]].round(3))

# Verificar convergencia
r_hats = summary['r_hat'].values
converged = all(r < 1.05 for r in r_hats)
print(f"\n{'✓' if converged else '✗'} Convergencia: {'OK' if converged else 'PROBLEMA'} (todos R-hat < 1.05)")

In [ ]:
# Paso 5-7: Scoring
print("Calculando anomaly scores...")

scores = steps.compute_scores(arrays['y'], trace, arrays['entity_idx'])
intervals = steps.compute_intervals(trace, arrays['entity_idx'])
scored_df = steps.create_scored_df(modeling_df, scores, intervals)

print(f"\n✓ Scored {len(scored_df)} observaciones")
print(f"\nTop 10 anomalías:")
print(scored_df[['entity_id', 'event_count', 'anomaly_score', 'has_attack']].head(10).to_string())

In [ ]:
# Paso 8: Evaluación
metrics = steps.evaluate(scored_df)

print("📈 Métricas de Evaluación:\n")
print(f"  PR-AUC:  {metrics['pr_auc']:.3f}")
print(f"  ROC-AUC: {metrics['roc_auc']:.3f}")
print(f"\n  Precision@10: {metrics.get('precision_at_10', 'N/A')}")
print(f"  Precision@25: {metrics.get('precision_at_25', 'N/A')}")
print(f"  Recall@10:    {metrics.get('recall_at_10', 'N/A')}")
print(f"  Recall@25:    {metrics.get('recall_at_25', 'N/A')}")

In [ ]:
# Visualización: Score Distribution por clase
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Score distribution
for label, group in scored_df.groupby('has_attack'):
    axes[0].hist(group['anomaly_score'], bins=30, alpha=0.6, 
                 label=f"{'Attack' if label else 'Normal'}", density=True)
axes[0].set_xlabel('Anomaly Score')
axes[0].set_ylabel('Density')
axes[0].set_title('Distribución de Scores por Clase')
axes[0].legend()

# PR Curve
from sklearn.metrics import precision_recall_curve
precision, recall, _ = precision_recall_curve(scored_df['has_attack'], scored_df['anomaly_score'])
axes[1].plot(recall, precision, 'b-', linewidth=2, label=f'BSAD (PR-AUC={metrics["pr_auc"]:.3f})')
axes[1].axhline(y=scored_df['has_attack'].mean(), color='r', linestyle='--', label='Random baseline')
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision-Recall Curve')
axes[1].legend()
axes[1].set_xlim([0, 1])
axes[1].set_ylim([0, 1])

plt.tight_layout()
plt.show()

In [ ]:
# Visualización: Top anomalías con incertidumbre
top_n = 15
top_anomalies = scored_df.head(top_n).copy()
top_anomalies['idx'] = range(top_n)

fig, ax = plt.subplots(figsize=(12, 6))

# Barras de score con error bars
colors = ['coral' if attack else 'steelblue' for attack in top_anomalies['has_attack']]
ax.barh(top_anomalies['idx'], top_anomalies['anomaly_score'], 
        xerr=top_anomalies['score_std'], color=colors, alpha=0.7, capsize=3)

# Labels
ax.set_yticks(top_anomalies['idx'])
ax.set_yticklabels([f"{row['entity_id']} (count={row['event_count']})" 
                    for _, row in top_anomalies.iterrows()])
ax.set_xlabel('Anomaly Score')
ax.set_title(f'Top {top_n} Anomalías con Intervalos de Incertidumbre')
ax.invert_yaxis()

# Legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='coral', label='Attack'),
                   Patch(facecolor='steelblue', label='Normal')]
ax.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.show()

print(f"\n🎯 De los top {top_n}: {top_anomalies['has_attack'].sum()} son ataques reales")

---
## Resumen para la Entrevista

### Elevator Pitch (30 segundos)

> "BSAD es un sistema de detección de anomalías para datos de seguridad que usa modelado Bayesiano jerárquico. 
> A diferencia de métodos clásicos, BSAD aprende un baseline personalizado para cada entidad (usuario, IP, etc.) 
> mientras comparte información entre entidades similares via partial pooling. Esto lo hace ideal para 
> detectar eventos raros como brute force o data exfiltration, donde el class imbalance es extremo 
> y necesitamos cuantificar la incertidumbre en nuestras predicciones."

### Puntos Clave

1. **Negative Binomial** para overdispersion (var > mean)
2. **Partial pooling** para entidades sparse
3. **PR-AUC** como métrica primaria (no ROC-AUC)
4. **Incertidumbre cuantificada** via posterior samples
5. **Triage module** para operacionalización (calibración de thresholds, enriquecimiento de alertas)

### Lo que NO hace BSAD

- No es real-time (MCMC es lento)
- No detecta ataques de contenido (SQLi, XSS)
- No maneja features continuos multivariados
- No funciona sin estructura entidad-tiempo